In [590]:
import numpy as np
import pandas as pd
import random
import copy

In [717]:
class ChessBoard:
    def __init__(self):
        self.letters = 'abcdefgh'
        self.letNum = {'a': 1,
                      'b': 2,
                      'c': 3,
                      'd': 4,
                      'e': 5,
                      'f': 6,
                      'g': 7,
                      'h': 8}
        self.numLet = {v: k for k,v in self.letNum.items()}
        self.numbers = np.arange(1,9)
        self.board = {}
        self.board_prior = {}
        self.pieceSymbol = {'empty': ' ',
                     'pawn': 'P',
                     'bishop': 'B',
                     'knight': 'N',
                     'rook': 'R',
                     'queen': 'Q',
                     'king': 'K'}
        self.pieceID = {'empty': 0,
                     'pawn': 1,
                     'bishop': 2,
                     'knight': 3,
                     'rook': 4,
                     'queen': 5,
                     'king': 6}
        self.pieceName = {v: k for k,v in self.pieceID.items()}
        self.pieceValues = {0: 0,
                           1: 1,
                           2: 3,
                           3: 3,
                           4: 5,
                           5: 9}
        self.game_start()
        self.side_to_move = 1
        
    def game_start(self):
        self.terminal_row_order = ['rook','knight','bishop','queen','king','bishop','knight','rook']
        self.board.update({f'{l}1': self.pieceID[self.terminal_row_order[p]] for p,l in enumerate(self.letters)})
        self.board.update({f'{l}2': self.pieceID['pawn'] for l in self.letters})
        self.board.update({f'{l}{n}': self.pieceID['empty'] for l in self.letters for n in self.numbers[2:6]})
        self.board.update({f'{l}7': self.pieceID['pawn']*-1 for l in self.letters})
        self.board.update({f'{l}8': self.pieceID[self.terminal_row_order[p]]*-1 for p,l in enumerate(self.letters)})
        self.board_prior = copy.deepcopy(self.board)
     
    def visualize_board(self):
        print("    ---------------------------------")
        for n in reversed(self.numbers):
            print(f"{n}   "
                 f"| {self.pieceSymbol[self.pieceName[abs(self.board[f'a{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'b{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'c{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'd{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'e{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'f{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'g{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board[f'h{n}'])]]} |")
            print("    ---------------------------------")
        print("                                     ")
        print("      a   b   c   d   e   f   g   h")
        
    def visualize_board_prior(self):
        print("    ---------------------------------")
        for n in reversed(self.numbers):
            print(f"{n}   "
                 f"| {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'a{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'b{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'c{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'd{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'e{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'f{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'g{n}'])]]} |"\
                 f" {self.pieceSymbol[self.pieceName[abs(self.board_prior[f'h{n}'])]]} |")
            print("    ---------------------------------")
        print("                                     ")
        print("      a   b   c   d   e   f   g   h")
    
    def pawn_moves(self,sq,c):
        l = sq[0]
        n = int(sq[1])
        moves = []
        # Move forward 1
        if self.board[f'{l}{n+c}'] == 0:
            moves.append(f'{l}{n+c}')
            # Move forward 2
            if self.board[f'{l}{n+2*c}'] == 0 and (n*c == 2 or n*c == -7) and self.board[sq] == self.board_prior[sq]:
                moves.append(f'{l}{n+2*c}')
        
        l_n = self.letNum[l]
        # Capture left diagonal
        if (1 <= l_n-c <=8) and self.board[f'{self.numLet[l_n-c]}{n+c}']*c < 0:
            moves.append(f'{self.numLet[l_n-c]}{n+c}')
        # Capture right diagonal
        if (1 <= l_n+c <=8) and self.board[f'{self.numLet[l_n+c]}{n+c}']*c < 0:
            moves.append(f'{self.numLet[l_n+c]}{n+c}')
        # En Passant (add later)
        if moves:    
            return moves
    
    def bishop_moves(self,sq,c):
        l = sq[0]
        n = int(sq[1])
        l_n = self.letNum[l]
        moves = []
        # Upper right
        temp_ln = l_n + c
        temp_n = n + c
        while (1 <= temp_ln <=8) and (1 <= temp_n <=8):
            if self.board[f'{self.numLet[temp_ln]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
            elif self.board[f'{self.numLet[temp_ln]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
                break
            elif self.board[f'{self.numLet[temp_ln]}{temp_n}']*c > 0:
                break
            temp_ln += c
            temp_n += c
        # Upper left
        temp_ln = l_n - c
        temp_n = n + c
        while (1 <= temp_ln <=8) and (1 <= temp_n <=8):
            if self.board[f'{self.numLet[temp_ln]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
            elif self.board[f'{self.numLet[temp_ln]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
                break
            elif self.board[f'{self.numLet[temp_ln]}{temp_n}']*c > 0:
                break
            temp_ln -= c
            temp_n += c
        # Lower right
        temp_ln = l_n + c
        temp_n = n - c
        while (1 <= temp_ln <=8) and (1 <= temp_n <=8):
            if self.board[f'{self.numLet[temp_ln]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
            elif self.board[f'{self.numLet[temp_ln]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
                break
            elif self.board[f'{self.numLet[temp_ln]}{temp_n}']*c > 0:
                break
            temp_ln += c
            temp_n -= c
        # Lower left
        temp_ln = l_n - c
        temp_n = n - c
        while (1 <= temp_ln <=8) and (1 <= temp_n <=8):
            if self.board[f'{self.numLet[temp_ln]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
            elif self.board[f'{self.numLet[temp_ln]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{temp_n}')
                break
            elif self.board[f'{self.numLet[temp_ln]}{temp_n}']*c > 0:
                break
            temp_ln -= c
            temp_n -= c
        if moves:
            return moves
    
    def knight_moves(self,sq,c):
        l = sq[0]
        n = int(sq[1])
        l_n = self.letNum[l]
        moves = []
        # Up/Down, right/Left
        for i in [1,-1]:
            for j in [2,-2]:
                if (1 <= l_n+i <=8) and (1 <= n+j <=8) and self.board[f'{self.numLet[l_n+i]}{n+j}']*c <= 0:
                    moves.append(f'{self.numLet[l_n+i]}{n+j}')
                else:
                    continue
        # Right/Left, up,down
        for i in [2,-2]:
            for j in [1,-1]:
                if (1 <= l_n+i <=8) and (1 <= n+j <=8) and self.board[f'{self.numLet[l_n+i]}{n+j}']*c <= 0:
                    moves.append(f'{self.numLet[l_n+i]}{n+j}')
                else:
                    continue
        if moves:
            return moves
    
    def rook_moves(self,sq,c):
        l = sq[0]
        n = int(sq[1])
        l_n = self.letNum[l]
        moves = []
        # Right
        temp_ln = l_n + c
        while (1 <= temp_ln <=8):
            if self.board[f'{self.numLet[temp_ln]}{n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{n}')
            elif self.board[f'{self.numLet[temp_ln]}{n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{n}')
                break
            elif self.board[f'{self.numLet[temp_ln]}{n}']*c > 0:
                break
            temp_ln += c
        # Left
        temp_ln = l_n - c
        while (1 <= temp_ln <=8):
            if self.board[f'{self.numLet[temp_ln]}{n}'] == 0:
                moves.append(f'{self.numLet[temp_ln]}{n}')
            elif self.board[f'{self.numLet[temp_ln]}{n}']*c < 0:
                moves.append(f'{self.numLet[temp_ln]}{n}')
                break
            elif self.board[f'{self.numLet[temp_ln]}{n}']*c > 0:
                break
            temp_ln -= c
        # Up
        temp_n = n + c
        while (1 <= temp_n <=8):
            if self.board[f'{self.numLet[l_n]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[l_n]}{temp_n}')
            elif self.board[f'{self.numLet[l_n]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[l_n]}{temp_n}')
                break
            elif self.board[f'{self.numLet[l_n]}{temp_n}']*c > 0:
                break
            temp_n += c
        # Down
        temp_n = n - c
        while (1 <= temp_n <=8):
            if self.board[f'{self.numLet[l_n]}{temp_n}'] == 0:
                moves.append(f'{self.numLet[l_n]}{temp_n}')
            elif self.board[f'{self.numLet[l_n]}{temp_n}']*c < 0:
                moves.append(f'{self.numLet[l_n]}{temp_n}')
                break
            elif self.board[f'{self.numLet[l_n]}{temp_n}']*c > 0:
                break
            temp_n -= c
        if moves:
            return moves
    
    def queen_moves(self,sq,c):
        moves = []
        b_moves = self.bishop_moves(sq,c)
        if b_moves:
            moves.append(b_moves)
        r_moves = self.rook_moves(sq,c)
        if r_moves:
            moves.append(r_moves)
        if len(moves) > 1:
            np.concatenate(moves)
        if moves:
            return moves
    
    def king_moves(self,sq,c):
        l = sq[0]
        n = int(sq[1])
        l_n = self.letNum[l]
        moves = []
        for i in [1,0,-1]:
            for j in [1,0,-1]:
                if (1 <= l_n+i <=8) and (1 <= n+j <=8) and self.board[f'{self.numLet[l_n+i]}{n+j}']*c <= 0:
                    moves.append(f'{self.numLet[l_n+i]}{n+j}')
                else:
                    continue
        if moves:
            return moves
    
    def gen_pseudolegal_moves(self,c,pieces_on_board=None):
        if c == self.side_to_move:
            pieces_on_board = []
            for sq,pID in self.board.items():
                if pID*c<=0:
                    continue
                else:
                    pieces_on_board.append([sq,pID])

        possible_moves = {}
        for sq,pID in pieces_on_board:
            # Pawn Moves
            if pID*c == 1:
                possible_moves[sq] = self.pawn_moves(sq,c)
            elif pID*c == 2:
                possible_moves[sq] = self.bishop_moves(sq,c)
            elif pID*c == 3:
                possible_moves[sq] = self.knight_moves(sq,c)
            elif pID*c == 4:
                possible_moves[sq] = self.rook_moves(sq,c)
            elif pID*c == 5:
                possible_moves[sq] = self.queen_moves(sq,c)
            elif pID*c == 6:
                possible_moves[sq] = self.king_moves(sq,c)
        possible_moves = {k: v for k,v in possible_moves.items() if v is not None}

        return possible_moves
    
    def is_King_in_check(self,c,board,opponent_pieces):
        rev_board = {v: k for k,v in board.items()}
        king_pos = rev_board[6*c]
        opponent_moves = self.gen_pseudolegal_moves(-c,opponent_pieces)
        if king_pos in opponent_moves.values():
            return True
        else:
            return False
        
    
    def check_legality(self,possible_moves):
        legal_moves = {}
        # Gen list of all opponent pieces
        c = self.side_to_move
        opponent_pieces_on_board = []
        for sq,pID in self.board.items():
            if pID*c>=0:
                continue
            else:
                opponent_pieces_on_board.append([sq,pID])
                
        for start,end in possible_moves.items():
            tmp_board = copy.deepcopy(self.board)
            tmp_board.update({f'{end}': self.board[f'{start}']})
            tmp_board.update({f'{start}': 0})
            if not self.is_King_in_check(self.side_to_move,tmp_board,opponent_pieces_on_board):
                legal_moves[f'{start}'] = end
            else:
                continue
            
        return legal_moves

    def random_move(self,legal_moves):
        random_key = random.choice(list(legal_moves.keys()))
        random_val = random.choice(list(legal_moves[random_key]))
        if self.side_to_move == 1:
            color = 'White'
        elif self.side_to_move == -1:
            color = 'Black'
        print(f'{color} plays {self.pieceSymbol[self.pieceName[abs(self.board[random_key])]]}{random_key} to {random_val}')
        self.board_prior = copy.deepcopy(self.board)
        self.board.update({f'{random_val}': self.board[f'{random_key}']})
        self.board.update({f'{random_key}': 0})
        self.side_to_move = -self.side_to_move
    
    def play(self,num_moves):
        #self.visualize_board()
        for _ in range(num_moves):
            pm = self.gen_pseudolegal_moves(self.side_to_move)
            lm = self.check_legality(pm)
            self.value_from_move(lm)
            print("\n")
            self.random_move(lm)
            self.visualize_board()
            print("\n\n\n\n\n")

    def value_from_move(self,legal_moves):
        lm_vals = copy.deepcopy(legal_moves)
        for piece,moves in legal_moves.items():
            piece_moves = []
            for move in moves:
                piece_moves.append(self.pieceValues[self.board[move]*-self.side_to_move])
            lm_vals[piece] = piece_moves


Next things to add:
 - Determine if opponent is in check
 - Determine checkmate
 - Determine stalemate
 - Minimax algorithm for choosing moves
 - Castling
 - En passant

In [718]:
game1 = ChessBoard()


In [719]:
game1.play(4)



White plays Nb1 to a3
    ---------------------------------
8   | R | N | B | Q | K | B | N | R |
    ---------------------------------
7   | P | P | P | P | P | P | P | P |
    ---------------------------------
6   |   |   |   |   |   |   |   |   |
    ---------------------------------
5   |   |   |   |   |   |   |   |   |
    ---------------------------------
4   |   |   |   |   |   |   |   |   |
    ---------------------------------
3   | N |   |   |   |   |   |   |   |
    ---------------------------------
2   | P | P | P | P | P | P | P | P |
    ---------------------------------
1   | R |   | B | Q | K | B | N | R |
    ---------------------------------
                                     
      a   b   c   d   e   f   g   h








Black plays Pf7 to f6
    ---------------------------------
8   | R | N | B | Q | K | B | N | R |
    ---------------------------------
7   | P | P | P | P | P |   | P | P |
    ---------------------------------
6   |   |   |   |   |   | P |   |   